In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as ex
import bz2
import torch
import _pickle as pickle
import scipy.stats as stats
import scipy
from plotly.subplots import make_subplots
import plotly.io as pio
import sys, os

# Global Variables

In [2]:
RESULT_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results"
REF_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/REF"
TRANS_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/TRANS"
REDUC_SAVE_PATH = "F:/Documents/MEX/Deep-learning-based-rig-agnostic-encoding/results/organized/REDUC"
METRICS = ["recon_error","adv_error", "rot_error", "delta_rot"]
METRICS_RENAME = ["Re. error", "Adv. error", "Rot. error", "RMS"]
SUFFIX = {"Raw":"_RAW", "Frozen":"_F", "Trained":"_T"}


# Functions

In [3]:
def pJoin(p1, p2):
    return os.path.join(p1,p2)

def load(file_path:str):
    with bz2.BZ2File(file_path, "rb") as f:
        obj = pickle.load(f)
    return obj
def save(file:object, file_path:str):
    with bz2.BZ2File(file_path, "w") as f:
        pickle.dump(file, f)
def mean_confidence_interval(data, confidence=0.95):
    data = np.nan_to_num(data, 0)
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def plot_bar(results:dict, title="", x_axis_name=None, y_axis_name=None, 
                    width=500, height=300, vertical=False,
                    add_hline=False, annotation_text="baseline", annotation_position="top"):

    if not vertical:
        fig = ex.bar(results, x="Models", y="Percentage", color="Metrics", opacity=0.8, text="Percentage", template="seaborn")
    else:
        fig = ex.bar(results, y="Models", x="Percentage", color="Metrics", opacity=0.8, text="Percentage", template="seaborn")

    if add_hline:
        if not vertical:
            fig.add_hline(y=400, x0=-0.5, x1=len(results["Models"])/4 - 0.5,
            line_width=2, line_dash="dash", line_color="black", opacity=0.6,
            annotation_text=annotation_text, annotation_position=annotation_position) 
        else:
            fig.add_vline(x=400, y0=-0.5, y1=len(results["Models"])/4 - 0.5,
            line_width=2, line_dash="dash", line_color="black", opacity=0.6,
            annotation_text=annotation_text, annotation_position=annotation_position) 

    fig.update_layout(
        title_text=title,
        width = width, height=height, 
        font_family="Serif", font_size=14, 
        margin_l=5, margin_t=40, margin_b=5, margin_r=5)

    if x_axis_name is not None:
        fig.update_xaxes(
            title_text=x_axis_name
        )
    if y_axis_name is not None:
        fig.update_yaxes(
            title_text=y_axis_name
        )
    return fig

def save_plot(plot:object, file_path="figure.svg"):
    plot.write_image(file_path)

def calc_diff(*results, multiplier=100, inverse=False):
    if len(results) < 2:
        raise ValueError("Need at least 2 arrays for comparison")
    if (len(results[0])) < 2:
        raise ValueError("The result array should contain results from each rig")
    if not inverse:
        comp_fn = lambda base,target: np.asarray([1 / (y / x) * multiplier for x,y in zip(base, target)], np.float64) 
    else:
        comp_fn = lambda base,target: np.asarray([(y / x) * multiplier for x,y in zip(base, target)], np.float64) 

    base = results[0]
    comparison = [[],[]]
    for target in results[1:]:
        comp_avg_over_rig = np.round(np.mean([comp_fn(b,t) for b,t in zip(base, target)]), decimals=2)
        pval_avg_over_rig = np.mean([stats.kruskal(t,b)[1] for b,t in zip(base, target)])
        comparison[0].append(comp_avg_over_rig)
        comparison[1].append(pval_avg_over_rig)
    return comparison



# Generation performance

## MoE vs LSTM

In [20]:
file_names = [] 
for _, _, names in os.walk(REF_SAVE_PATH):
    file_names = names
print(file_names)


['AE+LSTM.pbz2', 'AE+MoE.pbz2', 'DEC-CAT+MoE.pbz2', 'DEC-IN+MoE.pbz2', 'RBF-CAT+LSTM.pbz2', 'RBF-CAT+MoE.pbz2', 'RBF-IN+LSTM.pbz2', 'RBF-IN+MoE.pbz2', 'VAE-CAT+LSTM.pbz2', 'VAE-CAT+MoE.pbz2', 'VAE-IN+LSTM.pbz2', 'VAE-IN+MoE.pbz2']


In [68]:
comp_df = {"AE":{}, "RBF-IN":{}, "RBF-CAT":{}, "VAE-IN":{}, "VAE-CAT":{}}
for k in comp_df.keys():
    f1 = load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2"))
    f2 = load(pJoin(REF_SAVE_PATH,k+"+LSTM.pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [72]:
keys = list(comp_df.keys())
x = [k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Generation performance of LSTM compared to MoE", 
        width=600, add_hline=True, annotation_text="baseline")
save_plot(fig, "img/ref/MoE_vs_LSTM.svg")
fig.show()

In [40]:
for k in keys:
    for metric in METRICS:
        # comp_df[k][metric]= (comp_df[k][metric][0][0], comp_df[k][metric][1][0])
        comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/ref/MoE_vs_LSTM.csv")

                          AE           RBF-IN          RBF-CAT  \
recon_error     (31.94, 0.0)    (80.8, 0.003)     (30.85, 0.0)   
adv_error    (121.17, 0.093)  (112.29, 0.002)  (122.08, 0.171)   
rot_error       (75.04, 0.0)   (95.52, 0.035)     (77.88, 0.0)   
delta_rot       (58.85, 0.0)     (79.34, 0.0)     (57.48, 0.0)   

                      VAE-IN          VAE-CAT  
recon_error     (58.47, 0.0)      (37.1, 0.0)  
adv_error      (115.84, 0.0)  (123.86, 0.007)  
rot_error       (94.63, 0.0)     (82.43, 0.0)  
delta_rot    (148.03, 0.034)     (64.64, 0.0)  


## FC-CAT vs FC-IN

In [73]:
comp_df = {"RBF":{}, "VAE":{}, "DEC":{}}
for k in comp_df.keys():
    f1 = load(pJoin(REF_SAVE_PATH,k+"-IN+MoE.pbz2"))
    f2 = load(pJoin(REF_SAVE_PATH,k+"-CAT+MoE.pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [76]:
keys = list(comp_df.keys())
x = [k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Generation performance of FC-CAT compared to FC-IN", 
        width=600, add_hline=True, annotation_text="")
save_plot(fig, "img/ref/FC-IN_vs_FC-CAT.svg")
fig.show()

In [55]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= (comp_df[k][metric][0][0], np.round(comp_df[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/ref/FC-IN_vs_FC-CAT.csv")

                         RBF             VAE              DEC
recon_error    (397.77, 0.0)   (247.53, 0.0)  (172.77, 0.117)
adv_error    (101.55, 0.002)  (98.85, 0.272)  (104.77, 0.025)
rot_error      (140.29, 0.0)   (129.39, 0.0)  (111.71, 0.062)
delta_rot      (143.19, 0.0)   (260.38, 0.0)    (138.36, 0.0)


## FE vs FC-CAT

In [77]:
comp_df = {"RBF-CAT":{}, "VAE-CAT":{}, "DEC-CAT":{}}
f1 = load(pJoin(REF_SAVE_PATH,"AE+MoE.pbz2"))
for k in comp_df.keys():
    f2 = load(pJoin(REF_SAVE_PATH,k+"+MoE.pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [78]:
file_name = "FE_vs_FC-CAT"
keys = list(comp_df.keys())
x = [k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Generation performance of FC-CAT compared to FE", 
        width=600, add_hline=True, annotation_text="")
save_plot(fig, "img/ref/"+file_name+".svg")
fig.show()

In [64]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= (comp_df[k][metric][0][0], np.round(comp_df[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/ref/"+file_name+".csv")

                     RBF-CAT          VAE-CAT          DEC-CAT
recon_error  (112.49, 0.004)  (100.92, 0.123)      (48.6, 0.0)
adv_error    (100.07, 0.498)  (100.25, 0.409)  (106.26, 0.112)
rot_error    (102.46, 0.377)    (97.24, 0.09)   (82.33, 0.056)
delta_rot     (99.42, 0.454)   (84.77, 0.019)   (91.69, 0.183)


# Transferred quality
## FE Transferred (frozen, trained) vs FE Reference 

In [4]:
file_names = [] 
for _, _, names in os.walk(TRANS_SAVE_PATH):
    file_names = names
print(file_names)

['AE+MoE_F.pbz2', 'AE+MoE_RAW.pbz2', 'AE+MoE_T.pbz2', 'DEC-CAT+MoE_F.pbz2', 'DEC-CAT+MoE_RAW.pbz2', 'DEC-CAT+MoE_T.pbz2', 'DEC-IN+MoE_F.pbz2', 'DEC-IN+MoE_RAW.pbz2', 'DEC-IN+MoE_T.pbz2', 'RBF-CAT+MoE_F.pbz2', 'RBF-CAT+MoE_RAW.pbz2', 'RBF-CAT+MoE_T.pbz2', 'RBF-IN+MoE_F.pbz2', 'RBF-IN+MoE_RAW.pbz2', 'RBF-IN+MoE_T.pbz2', 'VAE-CAT+MoE_F.pbz2', 'VAE-CAT+MoE_RAW.pbz2', 'VAE-CAT+MoE_T.pbz2', 'VAE-IN+MoE_F.pbz2', 'VAE-IN+MoE_RAW.pbz2', 'VAE-IN+MoE_T.pbz2']


In [15]:
comp_df = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "AE+MoE"
f1 = load(pJoin(REF_SAVE_PATH, base+".pbz2"))
for k in comp_df.keys():
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+suffix+".pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric][1:], f2[metric])

In [7]:
file_name = "FE_RAW_F_T"
keys = list(comp_df.keys())
x = [k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Transferring quality of FE", 
        width=600, add_hline=True, annotation_text="Reference perf.")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [8]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= (comp_df[k][metric][0][0], np.round(comp_df[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/trans/"+file_name+".csv")

                         Raw           Frozen          Trained
recon_error     (39.37, 0.0)      (76.8, 0.0)     (67.99, 0.0)
adv_error       (91.32, 0.0)   (98.75, 0.124)    (98.0, 0.053)
rot_error       (72.34, 0.0)     (89.96, 0.0)     (88.76, 0.0)
delta_rot    (116.32, 0.001)  (100.85, 0.105)  (105.56, 0.128)


In [16]:
comp_df = {"Frozen":{}, "Trained":{}}
base = "AE+MoE"
f1 = load(pJoin(TRANS_SAVE_PATH, base+"_RAW.pbz2"))
for k in comp_df.keys():
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+suffix+".pbz2"))
    for metric in METRICS:
        comp_df[k][metric] = calc_diff(f1[metric], f2[metric])

In [17]:
file_name = "FE_F_T"
keys = list(comp_df.keys())
x = [k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df[k][metric][0][0] for metric in METRICS]
df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys))
fig = plot_bar(results=df_formatted, title="Performance improvment of FE", 
        width=600, add_hline=True, annotation_text="Cold perf.", annotation_position="bottom")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [18]:
for k in keys:
    for metric in METRICS:
        comp_df[k][metric]= (comp_df[k][metric][0][0], np.round(comp_df[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd = pd.DataFrame(comp_df)
print(comp_pd)
comp_pd.to_csv("tables/trans/"+file_name+".csv")

                    Frozen         Trained
recon_error  (198.31, 0.0)   (184.09, 0.0)
adv_error    (108.17, 0.0)   (107.91, 0.0)
rot_error    (126.17, 0.0)   (124.48, 0.0)
delta_rot     (92.56, 0.0)  (96.05, 0.052)


## FC
### RBF

In [19]:
comp_df_in = {"Raw":{}, "Frozen":{}, "Trained":{}}
comp_df_cat = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "RBF"
for k in comp_df_in.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-IN+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric][1:], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric][1:], f2[metric])


In [13]:
file_name = "RBF_RAW_F_T"
keys = list(comp_df_in.keys())
x = ["RBF-IN_" + k for k in keys for i in range(4)] + ["RBF-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Transferring quality of FC - RBF", vertical=True,
        width=800, height=600, add_hline=True, annotation_text="")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [14]:
for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                        Raw          Frozen          Trained
recon_error    (92.04, 0.0)    (94.97, 0.0)  (103.04, 0.005)
adv_error      (87.76, 0.0)    (96.67, 0.0)   (98.79, 0.045)
rot_error    (92.15, 0.102)  (91.47, 0.128)    (95.0, 0.003)
delta_rot     (168.08, 0.0)  (97.49, 0.442)     (98.61, 0.0)
                       Raw          Frozen          Trained
recon_error   (39.83, 0.0)    (67.86, 0.0)      (63.6, 0.0)
adv_error     (91.54, 0.0)  (97.52, 0.101)    (96.5, 0.098)
rot_error     (71.36, 0.0)    (86.86, 0.0)     (84.66, 0.0)
delta_rot    (107.72, 0.2)    (98.8, 0.22)  (100.62, 0.038)


In [117]:
# Against RAW

comp_df_in = {"Frozen":{}, "Trained":{}}
comp_df_cat = {"Frozen":{}, "Trained":{}}
base = "RBF"
for k in comp_df_in.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-IN+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-CAT+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric], f2[metric])


In [119]:
file_name = "RBF_F_T"
keys = list(comp_df_in.keys())
x = ["RBF-IN_" + k for k in keys for i in range(4)] + ["RBF-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Performance improvment of FC - RBF", vertical=True,
        width=800, height=600, add_hline=True, annotation_text="Cold perf.", annotation_position="top right")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

In [120]:
for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                      Frozen         Trained
recon_error  (103.99, 0.287)  (114.8, 0.001)
adv_error    (112.96, 0.008)   (114.04, 0.0)
rot_error     (100.0, 0.178)   (103.87, 0.0)
delta_rot       (62.72, 0.0)    (62.95, 0.0)
                    Frozen         Trained
recon_error  (171.43, 0.0)   (166.63, 0.0)
adv_error    (107.63, 0.0)   (106.47, 0.0)
rot_error    (122.57, 0.0)   (120.62, 0.0)
delta_rot     (93.68, 0.0)  (96.56, 0.111)


### VAE

In [20]:
comp_df_in = {"Raw":{}, "Frozen":{}, "Trained":{}}
comp_df_cat = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "VAE"
for k in comp_df_in.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-IN+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric][1:], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric][1:], f2[metric])

file_name = base+"_RAW_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(4)] + [base+"-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Transferring quality of FC - "+base, vertical=True,
        width=800, height=600, add_hline=True, annotation_text="")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                       Raw           Frozen          Trained
recon_error    (63.2, 0.0)     (81.66, 0.0)     (97.45, 0.0)
adv_error     (81.63, 0.0)     (92.35, 0.0)   (94.49, 0.001)
rot_error     (88.35, 0.0)      (90.9, 0.0)   (94.98, 0.208)
delta_rot    (259.67, 0.0)  (127.69, 0.167)  (111.81, 0.034)
                      Raw          Frozen          Trained
recon_error  (48.76, 0.0)    (78.43, 0.0)   (87.41, 0.005)
adv_error    (95.73, 0.0)  (94.79, 0.006)   (99.51, 0.059)
rot_error    (80.34, 0.0)  (91.52, 0.002)   (98.04, 0.038)
delta_rot    (61.05, 0.0)    (84.23, 0.0)  (101.13, 0.023)


In [125]:
# Against RAW
comp_df_in = {"Frozen":{}, "Trained":{}}
comp_df_cat = {"Frozen":{}, "Trained":{}}
base = "VAE"
for k in comp_df_in.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-IN+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-CAT+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric], f2[metric])

file_name = base+"_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(4)] + [base+"-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Performance improvement of FC - "+base, vertical=True,
        width=800, height=600, add_hline=True, annotation_text="Cold perf.", annotation_position="top right")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                      Frozen          Trained
recon_error   (132.7, 0.002)     (164.4, 0.0)
adv_error      (115.71, 0.0)    (119.37, 0.0)
rot_error    (103.16, 0.222)  (108.55, 0.001)
delta_rot       (55.59, 0.0)     (50.47, 0.0)
                    Frozen        Trained
recon_error  (167.02, 0.0)  (183.63, 0.0)
adv_error     (99.76, 0.0)  (104.43, 0.0)
rot_error    (115.13, 0.0)  (123.69, 0.0)
delta_rot    (147.09, 0.0)  (172.85, 0.0)


In [22]:
comp_df_in = {"Raw":{}, "Frozen":{}, "Trained":{}}
comp_df_cat = {"Raw":{}, "Frozen":{}, "Trained":{}}
base = "DEC"
for k in comp_df_in.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-IN+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric][1:], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric][1:], f2[metric])

file_name = base+"_RAW_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(4)] + [base+"-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Transferring quality of FC - "+base, vertical=True,
        width=800, height=600, add_hline=True, annotation_text="")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                        Raw         Frozen         Trained
recon_error    (91.07, 0.0)   (91.48, 0.0)  (107.1, 0.006)
adv_error      (86.76, 0.0)  (96.27, 0.15)    (98.2, 0.23)
rot_error    (92.55, 0.218)   (91.43, 0.0)  (98.89, 0.032)
delta_rot      (183.5, 0.0)  (105.28, 0.0)   (111.1, 0.25)
                         Raw          Frozen          Trained
recon_error     (79.26, 0.0)  (80.03, 0.017)    (114.51, 0.0)
adv_error     (90.51, 0.003)  (87.04, 0.017)   (95.39, 0.068)
rot_error     (89.79, 0.002)    (83.46, 0.0)     (98.22, 0.0)
delta_rot    (138.35, 0.198)  (89.98, 0.003)  (103.73, 0.471)


In [127]:
# Against RAW
comp_df_in = {"Frozen":{}, "Trained":{}}
comp_df_cat = {"Frozen":{}, "Trained":{}}
base = "DEC"
for k in comp_df_in.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-IN+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-IN+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_in[k][metric] = calc_diff(f1[metric], f2[metric])

for k in comp_df_cat.keys():
    f1 = load(pJoin(TRANS_SAVE_PATH, base+"-CAT+MoE_RAW.pbz2"))
    suffix = SUFFIX[k]
    f2 = load(pJoin(TRANS_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
    for metric in METRICS:
        comp_df_cat[k][metric] = calc_diff(f1[metric], f2[metric])

file_name = base+"_F_T"
keys = list(comp_df_in.keys())
x = [base+"-IN_" + k for k in keys for i in range(4)] + [base+"-CAT_" + k for k in keys for i in range(4)]
y = []
for k in keys:
    y += [comp_df_in[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_cat[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*2)
fig = plot_bar(results=df_formatted, title="Performance improvement of FC - "+base, vertical=True,
        width=800, height=600, add_hline=True, annotation_text="Cold perf.", annotation_position="top right")
save_plot(fig, "img/trans/"+file_name+".svg")
fig.show()

for k in keys:
    for metric in METRICS:
        comp_df_in[k][metric]= (comp_df_in[k][metric][0][0], np.round(comp_df_in[k][metric][1][0], 3))
        comp_df_cat[k][metric]= (comp_df_cat[k][metric][0][0], np.round(comp_df_cat[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_in = pd.DataFrame(comp_df_in)
comp_pd_cat = pd.DataFrame(comp_df_cat)
print(comp_pd_in)
print(comp_pd_cat)
comp_pd_in.to_csv("tables/trans/"+file_name+"_in.csv")
comp_pd_cat.to_csv("tables/trans/"+file_name+"_cat.csv")

                      Frozen        Trained
recon_error  (102.76, 0.048)   (118.7, 0.0)
adv_error      (112.94, 0.0)  (114.53, 0.0)
rot_error     (99.66, 0.001)  (107.57, 0.0)
delta_rot       (64.54, 0.0)   (66.62, 0.0)
                     Frozen          Trained
recon_error    (98.18, 0.0)    (145.41, 0.0)
adv_error       (96.9, 0.0)  (106.07, 0.238)
rot_error    (94.55, 0.226)    (111.2, 0.07)
delta_rot      (66.84, 0.0)     (78.49, 0.0)


## REDUC TRANSFERRING QUALITY

In [23]:
file_names = [] 
for _, _, names in os.walk(REDUC_SAVE_PATH):
    file_names = names
print(file_names)

['AE+MoE_F.pbz2', 'AE+MoE_T.pbz2', 'DEC-CAT+MoE_F.pbz2', 'DEC-CAT+MoE_T.pbz2', 'RBF-CAT+MoE_F.pbz2', 'RBF-CAT+MoE_T.pbz2', 'VAE-CAT+MoE_F.pbz2', 'VAE-CAT+MoE_T.pbz2']


In [24]:
comp_df_ae = {"Frozen":{}, "Trained":{}}
comp_df_rbf = {"Frozen":{}, "Trained":{}}
comp_df_vae = {"Frozen":{}, "Trained":{}}
comp_df_dec = {"Frozen":{}, "Trained":{}}

def fill_df(comp_df,base):
    for k in comp_df.keys():
        suffix = SUFFIX[k]
        if base == "AE":
            f1 = load(pJoin(REF_SAVE_PATH, base+"+MoE.pbz2"))
            f2 = load(pJoin(REDUC_SAVE_PATH,base+"+MoE"+suffix+".pbz2"))
        else:
            f1 = load(pJoin(REF_SAVE_PATH, base+"-CAT+MoE.pbz2"))
            f2 = load(pJoin(REDUC_SAVE_PATH,base+"-CAT+MoE"+suffix+".pbz2"))
            
        for metric in METRICS:
            comp_df[k][metric] = calc_diff(f1[metric][1:], f2[metric])

fill_df(comp_df_ae, "AE")
fill_df(comp_df_rbf, "RBF")
fill_df(comp_df_vae, "VAE")
fill_df(comp_df_dec, "DEC")



In [25]:
file_name = "reduc_RAW_F_T"
keys = list(comp_df_ae.keys())
x = ["AE_" + k for k in keys for i in range(4)] + \
    ["RBF-CAT_" + k for k in keys for i in range(4)] + \
    ["VAE-CAT_" + k for k in keys for i in range(4)] + \
    ["DEC-CAT_" + k for k in keys for i in range(4)]

y = []
for k in keys:
    y += [comp_df_ae[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_rbf[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_vae[k][metric][0][0] for metric in METRICS]
for k in keys:
    y += [comp_df_dec[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys)*4)
fig = plot_bar(results=df_formatted, title="Transferring quality of FS", vertical=True,
        width=800, height=600, add_hline=True, annotation_text="Reference perf.", annotation_position="top right")
save_plot(fig, "img/reduc/"+file_name+".svg")
fig.show()


In [26]:
for k in keys:
    for metric in METRICS:
        comp_df_ae[k][metric]= (comp_df_ae[k][metric][0][0], np.round(comp_df_ae[k][metric][1][0], 3))
        comp_df_rbf[k][metric]= (comp_df_rbf[k][metric][0][0], np.round(comp_df_rbf[k][metric][1][0], 3))
        comp_df_vae[k][metric]= (comp_df_vae[k][metric][0][0], np.round(comp_df_vae[k][metric][1][0], 3))
        comp_df_dec[k][metric]= (comp_df_dec[k][metric][0][0], np.round(comp_df_dec[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_ae = pd.DataFrame(comp_df_ae)
comp_pd_rbf = pd.DataFrame(comp_df_rbf)
comp_pd_vae = pd.DataFrame(comp_df_vae)
comp_pd_dec = pd.DataFrame(comp_df_dec)

print(comp_pd_ae)
print(comp_pd_rbf)
print(comp_pd_vae)
print(comp_pd_dec)
comp_pd_ae.to_csv("tables/reduc/"+file_name+"_ae.csv")
comp_pd_rbf.to_csv("tables/reduc/"+file_name+"_rbg.csv")
comp_pd_vae.to_csv("tables/reduc/"+file_name+"_vae.csv")
comp_pd_dec.to_csv("tables/reduc/"+file_name+"_dec.csv")

                     Frozen          Trained
recon_error    (66.92, 0.0)     (65.98, 0.0)
adv_error    (99.18, 0.001)  (101.85, 0.159)
rot_error     (104.89, 0.0)    (102.32, 0.0)
delta_rot    (99.23, 0.076)  (100.32, 0.024)
                      Frozen          Trained
recon_error     (40.14, 0.0)     (60.99, 0.0)
adv_error       (91.71, 0.0)   (98.05, 0.129)
rot_error       (93.07, 0.0)    (109.94, 0.0)
delta_rot    (105.67, 0.337)  (101.37, 0.234)
                   Frozen         Trained
recon_error  (46.47, 0.0)    (69.07, 0.0)
adv_error    (96.42, 0.0)  (98.63, 0.004)
rot_error    (85.98, 0.0)   (108.32, 0.0)
delta_rot    (51.79, 0.0)  (90.84, 0.166)
                      Frozen         Trained
recon_error     (62.11, 0.0)   (105.22, 0.0)
adv_error       (86.93, 0.0)  (93.54, 0.208)
rot_error      (110.41, 0.0)   (115.28, 0.0)
delta_rot    (154.89, 0.224)  (97.48, 0.012)


# Summary

In [37]:
comp_df_ae = {"AE_{FE}":{}, "AE_{FS}":{}}
comp_df_rbf = {"RBF_{FC}":{}, "RBF_{FS}":{}}
comp_df_vae = {"VAE_{FC}":{}, "VAE_{FS}":{}}
comp_df_dec = {"DEC_{FC}":{}, "DEC_{FS}":{}}

def fill_df(comp_df,base):
    for k in comp_df.keys():
        suffix = "_T"
        f1 = load(pJoin(REF_SAVE_PATH, "AE+MoE.pbz2"))
        path = REDUC_SAVE_PATH if "_{FS}" in k else TRANS_SAVE_PATH
        if base == "AE":
            f2 = load(pJoin(path,base+"+MoE"+suffix+".pbz2"))
        else:
            f2 = load(pJoin(path,base+"-CAT+MoE"+suffix+".pbz2"))
                
        for metric in METRICS:
            comp_df[k][metric] = calc_diff(f1[metric][1:], f2[metric])

fill_df(comp_df_ae, "AE")
fill_df(comp_df_rbf, "RBF")
fill_df(comp_df_vae, "VAE")
fill_df(comp_df_dec, "DEC")



In [38]:
file_name = "Summary"
keys_ae = list(comp_df_ae.keys())
keys_rbf = list(comp_df_rbf.keys())
keys_vae = list(comp_df_vae.keys())
keys_dec = list(comp_df_dec.keys())

x = [k for k in keys_ae for i in range(4)] + \
    [k for k in keys_rbf for i in range(4)] + \
    [k for k in keys_vae for i in range(4)] + \
    [k for k in keys_dec for i in range(4)]

y = []
for k in keys_ae:
    y += [comp_df_ae[k][metric][0][0] for metric in METRICS]
for k in keys_rbf:
    y += [comp_df_rbf[k][metric][0][0] for metric in METRICS]
for k in keys_vae:
    y += [comp_df_vae[k][metric][0][0] for metric in METRICS]
for k in keys_dec:
    y += [comp_df_dec[k][metric][0][0] for metric in METRICS]

df_formatted = dict(Models=x, Percentage=y, Metrics=METRICS_RENAME*len(keys_ae)*4)
fig = plot_bar(results=df_formatted, title="Summary - Transferred models compared to the reference perf. of AE+MoE", vertical=True,
        width=800, height=600, add_hline=True, annotation_text="Reference perf. of AE+MoE", annotation_position="top right")
save_plot(fig, "img/"+file_name+".svg")
fig.show()


In [39]:
for k in keys_ae:
    for metric in METRICS:
        comp_df_ae[k][metric]= (comp_df_ae[k][metric][0][0], np.round(comp_df_ae[k][metric][1][0], 3))
for k in keys_rbf:
    for metric in METRICS:
        comp_df_rbf[k][metric]= (comp_df_rbf[k][metric][0][0], np.round(comp_df_rbf[k][metric][1][0], 3))
for k in keys_vae:
    for metric in METRICS:
        comp_df_vae[k][metric]= (comp_df_vae[k][metric][0][0], np.round(comp_df_vae[k][metric][1][0], 3))
for k in keys_dec:
    for metric in METRICS:
        comp_df_dec[k][metric]= (comp_df_dec[k][metric][0][0], np.round(comp_df_dec[k][metric][1][0], 3))
        # comp_df[k][metric]= (comp_df[k][metric][0], np.round(comp_df[k][metric][1], 3))

comp_pd_ae = pd.DataFrame(comp_df_ae)
comp_pd_rbf = pd.DataFrame(comp_df_rbf)
comp_pd_vae = pd.DataFrame(comp_df_vae)
comp_pd_dec = pd.DataFrame(comp_df_dec)

print(comp_pd_ae)
print(comp_pd_rbf)
print(comp_pd_vae)
print(comp_pd_dec)
comp_pd_ae.to_csv("tables/"+file_name+"_ae.csv")
comp_pd_rbf.to_csv("tables/"+file_name+"_rbg.csv")
comp_pd_vae.to_csv("tables/"+file_name+"_vae.csv")
comp_pd_dec.to_csv("tables/"+file_name+"_dec.csv")

                     AE_{FE}          AE_{FS}
recon_error     (67.99, 0.0)     (65.98, 0.0)
adv_error      (98.0, 0.053)  (101.85, 0.159)
rot_error       (88.76, 0.0)    (102.32, 0.0)
delta_rot    (105.56, 0.128)  (100.32, 0.024)
                   RBF_{FC}        RBF_{FS}
recon_error    (68.55, 0.0)    (67.48, 0.0)
adv_error    (96.87, 0.109)  (98.51, 0.072)
rot_error      (85.64, 0.0)    (109.2, 0.0)
delta_rot    (99.81, 0.002)  (100.6, 0.121)
                    VAE_{FC}        VAE_{FS}
recon_error     (79.85, 0.0)    (64.68, 0.0)
adv_error    (100.24, 0.091)  (99.36, 0.224)
rot_error       (93.84, 0.0)   (104.37, 0.0)
delta_rot       (86.01, 0.0)    (77.45, 0.0)
                   DEC_{FC}      DEC_{FS}
recon_error    (48.91, 0.0)  (44.09, 0.0)
adv_error      (93.74, 0.0)  (91.85, 0.0)
rot_error      (77.42, 0.0)  (96.52, 0.0)
delta_rot    (92.11, 0.129)  (86.42, 0.0)
